In [17]:
import pandas as pd
from gurobipy import *
import networkx as nx

In [2]:
DtD = pd.read_excel("D_DtD.xlsx")
DtS = pd.read_excel("D_DtS.xlsx")
StS = pd.read_excel("D_StS.xlsx")
P_Ds = pd.read_excel("P_D.xlsx")
Pot_loc = pd.read_excel("Pot_locs.xlsx")
Supplier = pd.read_excel("Supplier.xlsx")
DtD = DtD.iloc[:-2,:-2]

In [3]:
#Sets
S = {Supplier.ID[i]: {'ID':Supplier.ID[i],
                      'X':Supplier.X[i],
                      'Y':Supplier.Y[i],
                      'Product_Type':Supplier.Product[i], 
                      'Capacity':Supplier.Prod_Vol[i],
                      'Vehicle':Supplier.V_type[i],
                     } for i in range(len(Supplier))}

W = {Pot_loc.P_D[i]:{'X':Pot_loc.X[i],
                   'Y':Pot_loc.Y[i],
                   'Cost':Pot_loc.C[i],
                   'Postal_District':Pot_loc.P_D[i],
                    'Capacity':Pot_loc.Q[i],
                   } for i in range(len(Pot_loc))}

C = {P_Ds.P_D[i]:{'Postal_District':P_Ds.P_D[i],
                 'Demand_P1': P_Ds.G1[i],
                 'Demand_P2': P_Ds.G2[i],
                 'Demand_P3': P_Ds.G3[i],
                 'Demand_P4': P_Ds.G4[i],
                } for i in range(len(P_Ds))}

P = {1,2,3,4}

In [4]:
unitcost = dict()
for supplier in S:
    if S[supplier]['Vehicle'] == 1:
        unitcost[supplier] = 0.09
    if S[supplier]['Vehicle'] == 2:
        unitcost[supplier] = 0.226

In [5]:
Dtd = DtD.set_index('Postal_District').unstack()
Dts = DtS.set_index('Postal_District').unstack()
Sts = StS.set_index('Name').unstack()

In [6]:
Dmax = {c : max(Dtd[c]) for c in W}

In [7]:
can_serve = {w:Dtd[w].loc[Dtd[w]<=150].index for w in W}

In [8]:
can_serve['AB11']

Index(['AB10', 'AB11', 'AB12', 'AB13', 'AB14', 'AB15', 'AB16', 'AB21', 'AB22',
       'AB23',
       ...
       'PH22', 'PH23', 'PH24', 'PH25', 'PH26', 'PH30', 'PH31', 'PH32', 'PH33',
       'PH34'],
      dtype='object', name='Postal_District', length=241)

In [9]:
#Model mockup
m = Model('Assignment')
m.Params.OutputFlag = 1
m.Params.IntFeasTol = 1e-6

#Vars
z = m.addVars([i for i in W],
              vtype = GRB.BINARY,
             name = 'z')
x = m.addVars([(w,c)  for w in can_serve for c in can_serve[w]],
             vtype = GRB.BINARY,
             name = 'x')
f = m.addVars([(p,w,c) for w in can_serve for c in can_serve[w] for p in P],
             vtype = GRB.CONTINUOUS,
             name = 'f')
F = m.addVars([(w,s) for w in W for s in S],
             vtype = GRB.CONTINUOUS,
             name = 'F')
a = m.addVars([c for c in C],
             vtype = GRB.BINARY,
             name = 'a')
dmin = m.addVars([c for c in C],
                vtype = GRB.CONTINUOUS,
                name = 'dmin')
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-10-13
Set parameter IntFeasTol to value 1e-06


In [10]:
#Constraints
c1 = m.addConstrs((quicksum(x[w,c] for w in W if (w,c) in x) == 1 for c in C),
                 name = 'Warehouse_client_assignment')

c2 = m.addConstrs((f[p,w,c]-C[c]['Demand_P%i'%p]*x[w,c] >= 0 for w in can_serve for c in can_serve[w] for p in P),
                 name = 'Cons2')

c3 = m.addConstrs((quicksum(F[w,s] for s in S)-quicksum(f[p,w,c] for c in C if (w,c) in f) == 0 for w in W for p in P),
                 name = 'Cons3')

c4 = m.addConstrs((quicksum(F[w,s] for s in S) - W[w]['Capacity']*z[w] <= 0 for w in W),
                 name = 'Warehouse_Capacity')

c5 = m.addConstrs((quicksum(F[w,s] for w in W) <= S[s]['Capacity'] for s in S),
                 name = 'Supplier_Volume')

c6 = m.addConstrs((x[w,c]-z[w] <= 0 for w in can_serve for c in can_serve[w]),
                 name = 'Cons6')

c7 = m.addConstrs((dmin[c] - Dtd[w][c]*z[w]-Dmax[c]*(1-z[w]) <= 0 for c in C for w in W),
                 name = 'Cons7')

c8 = m.addConstrs((quicksum(Dtd[w][c]*x[w,c] for w in W if (w,c) in x)- dmin[c] <= Dmax[c]*a[c] for c in C),
                 name = 'Cons8')
m.update()

In [11]:
#Objective funct
m.setObjective(quicksum(0.13*f[p,w,c] for p in P for w in can_serve for c in can_serve[w]) + quicksum(unitcost[s]*Dts[s][w]*F[w,s] for w in W for s in S)+quicksum(W[w]['Cost']*z[w] for w in W)+50000*quicksum(a[c] for c in C)
              , GRB.MINIMIZE)
m.update()
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 768883 rows, 596790 columns and 1900856 nonzeros
Model fingerprint: 0xa79f9519
Variable types: 481480 continuous, 115310 integer (115310 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+07]
  Objective range  [4e-02, 4e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+06]
Found heuristic solution: objective 6.535976e+08
Presolve removed 460216 rows and 481040 columns (presolve time = 7s) ...
Presolve removed 460216 rows and 481040 columns (presolve time = 13s) ...
Presolve removed 460216 rows and 481040 columns
Presolve time: 13.39s
Presolved: 308667 rows, 115750 columns, 844874 nonzeros
Variable types: 440 continuous, 115310 integer (115310 binary)
Found heuristic solution: objective 6.534976e+08

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log on

In [14]:
cont = 0
for i in z:
    if z[i].x == 1:
        print(i)

FK20
HS3
IV24


In [13]:
for (i,j) in x:
    if x[i,j].x == 1:
        print(i,j)

FK20 AB10
FK20 AB11
FK20 AB12
FK20 AB13
FK20 AB14
FK20 AB15
FK20 AB25
FK20 AB30
FK20 AB31
FK20 AB35
FK20 AB39
FK20 DD1
FK20 DD2
FK20 DD3
FK20 DD4
FK20 DD5
FK20 DD6
FK20 DD7
FK20 DD8
FK20 DD9
FK20 DD10
FK20 DD11
FK20 DG1
FK20 DG2
FK20 DG3
FK20 DG4
FK20 DG5
FK20 DG6
FK20 DG7
FK20 DG8
FK20 DG9
FK20 DG10
FK20 DG11
FK20 DG12
FK20 DG13
FK20 DG14
FK20 EH1
FK20 EH2
FK20 EH3
FK20 EH4
FK20 EH5
FK20 EH6
FK20 EH7
FK20 EH8
FK20 EH9
FK20 EH10
FK20 EH11
FK20 EH12
FK20 EH13
FK20 EH14
FK20 EH15
FK20 EH16
FK20 EH17
FK20 EH18
FK20 EH19
FK20 EH20
FK20 EH21
FK20 EH22
FK20 EH23
FK20 EH24
FK20 EH25
FK20 EH26
FK20 EH27
FK20 EH28
FK20 EH29
FK20 EH30
FK20 EH31
FK20 EH32
FK20 EH33
FK20 EH34
FK20 EH35
FK20 EH36
FK20 EH37
FK20 EH38
FK20 EH39
FK20 EH40
FK20 EH41
FK20 EH42
FK20 EH43
FK20 EH44
FK20 EH45
FK20 EH46
FK20 EH47
FK20 EH48
FK20 EH49
FK20 EH51
FK20 EH52
FK20 EH53
FK20 EH54
FK20 EH55
FK20 FK1
FK20 FK2
FK20 FK3
FK20 FK4
FK20 FK5
FK20 FK6
FK20 FK7
FK20 FK8
FK20 FK9
FK20 FK10
FK20 FK11
FK20 FK12
FK20 FK13
FK20 F

In [16]:
W['FK20']['X']

236259

In [19]:
G = nx.graph()

TypeError: 'module' object is not callable